In [40]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 
import plotly.express as px 


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>
/kaggle/input/fake-news-detection-using-machine-learning/News.csv


Loading Dataset & Preprocessing 

In [24]:
news_df = pd.read_csv("/kaggle/input/fake-news-detection-using-machine-learning/News.csv", index_col=0)
news_df.head()

,title,text,subject,date,class
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017",0
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017",0
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017",0
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017",0
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017",0


In [25]:
news_df.shape

(44919, 5)

In [26]:
# Droppig title, subject, and date columns since they are useless for determining validity of the news report
news_df = news_df.drop(["title", "subject", "date"], axis=1)

In [27]:
news_df.head()

,text,class
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0


In [28]:
news_df.isnull().sum()

text     0
class    0
dtype: int64

In [72]:
# Data Shuffling to avoid predictability 
news_df = news_df.sample(frac=1, random_state=42).reset_index(drop=True)
news_df

,text,class
0,jennifer garner given donald trump washingtont...,0
1,cannes france reuters us president donald trum...,1
2,riiiight choose works,0
3,brussels reuters european union leaders thursd...,1
4,washington reuters senior us chinese officials...,1
...,...,...
44914,reuters a new jersey judge plans decide friday...,1
44915,washington reuters white house officials said ...,1
44916,in 2016 214 million people moved germany 2015 ...,0
44917,and thought terrorism thing needed concerned t...,0


In [39]:
# Checking the representation of true news articles and fake news articles in the dataset 
fig = px.histogram(news_df, x="class", color="class").show()


In [44]:
!pip list | grep re #NLTK is installed

aiobotocore                        2.17.0
albucore                           0.0.19
array_record                       0.5.1
audioread                          3.0.1
botocore                           1.35.93
Deprecated                         1.2.15
dm-tree                            0.1.8
docker-pycreds                     0.4.0
eerepr                             0.0.4
en-core-web-sm                     3.7.1
fastcore                           1.7.27
fastprogress                       1.0.3
featuretools                       1.31.0
firebase-admin                     6.6.0
future                             1.0.0
google-api-core                    1.34.1
google-cloud-core                  2.4.1
google-cloud-datastore             2.20.2
google-cloud-firestore             2.19.0
google-cloud-resource-manager      1.14.0
google-resumable-media             2.7.2
greenlet                           3.1.1
gspread                            6.0.2
gspread-dataframe                  3.3.1
httpc

In [45]:
from tqdm import tqdm
import re
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.porter import PorterStemmer
from wordcloud import WordCloud

[nltk_data] Error loading punkt: <urlopen error [Errno -3] Temporary
[nltk_data]     failure in name resolution>
[nltk_data] Error loading stopwords: <urlopen error [Errno -3]
[nltk_data]     Temporary failure in name resolution>


In [47]:
# Function for processing text(removing unnecessary characters, lowercasing text, removing stopwords)
def preprocess_text(text_data):
    preprocessed_text = []
    
    for sentence in tqdm(text_data):
        sentence = re.sub(r'[^\w\s]', '', sentence) # Keeping any word characters and whitespace characters
        preprocessed_text.append(' '.join(token.lower() # Lowercasing all text
                                  for token in str(sentence).split() # Splitting sentence based on whitespace
                                  if token not in stopwords.words('english'))) # Removing English stopwords

    return preprocessed_text

In [49]:
# Applying processing function to each news article
preprocessed_review = preprocess_text(news_df['text'].values) 
news_df['text'] = preprocessed_review

100%|██████████| 44919/44919 [35:31<00:00, 21.08it/s] 


In [70]:
news_df #Verifying processed text

,text,class
0,washington reuters us senate republicans made ...,1
1,detroit reuters detroit may sue consultants wo...,1
2,geneva reuters the un human rights council tue...,1
3,do know he edward ed mezvinsky born january 17...,0
4,washington reuters president barack obamas adm...,1
...,...,...
44914,according the sun george w bush successor bara...,0
44915,watch barack obama identify leader black lives...,0
44916,chicago reuters a growing number us states ref...,1
44917,moscow reuters the kremlin said thursday us pr...,1


In [100]:
# Splitting dataset into train,val, and test sets(60-20-20 split)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression  

x_train, x_test, y_train, y_test = train_test_split(news_df["text"], news_df["class"], test_size=0.25)

In [101]:
print(f"Training Set Shape: x_train: {x_train.shape}, y_train: {y_train.shape}") 
print(f"Test Set Shape: x_train: {x_test.shape}, y_train: {y_test.shape}") 

Training Set Shape: x_train: (33689,), y_train: (33689,)
Test Set Shape: x_train: (11230,), y_train: (11230,)


In [102]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
x_train = vectorizer.fit_transform(x_train)
x_test = vectorizer.transform(x_test)

In [104]:
print(x_train.shape) 
print(x_test.shape) 

(33689, 195976)
(11230, 195976)


In [105]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression()
model.fit(x_train, y_train)

# testing the model
print(accuracy_score(y_train, model.predict(x_train)))
print(accuracy_score(y_test, model.predict(x_test)))

0.9936180949271276
0.9909171861086375
